# Test de Indicadores Técnicos Básicos

Este notebook demuestra el uso de los indicadores técnicos básicos del módulo.

## Objetivo

Mostrar cómo:
1. Cargar y limpiar datos
2. Calcular indicadores de diferentes categorías
3. Visualizar resultados
4. Interpretar los indicadores

**NOTA**: Este notebook solo calcula indicadores. No genera señales de trading.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Añadir src al path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Importar módulos del proyecto
from src.data import DataProvider, DataCleaner
from src.analisis.technical.indicators import trend, momentum, volatility, volume, price_action

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Cargar y Limpiar Datos

In [ ]:
# Crear provider y obtener datos
provider = DataProvider()
symbol = "AAPL"
print(f"Descargando datos para {symbol}...")

raw_data = provider.get_price_data(symbol, period="1y", interval="1d")
print(f"Datos descargados: {len(raw_data)} períodos")
print(f"Rango de fechas: {raw_data.index.min()} a {raw_data.index.max()}")

# Limpiar datos
cleaner = DataCleaner()
cleaned_data, metadata = cleaner.clean_price_data(raw_data, symbol=symbol)
print(f"\nDatos limpiados. Forma final: {cleaned_data.shape}")
print(f"Columnas disponibles: {list(cleaned_data.columns)}")

## 2. Indicadores de Tendencia

### SMA y EMA

In [ ]:
# Calcular SMAs y EMAs
df = trend.calculate_sma(cleaned_data.copy(), period=20, output_column='SMA_20')
df = trend.calculate_sma(df, period=50, output_column='SMA_50')
df = trend.calculate_ema(df, period=12, output_column='EMA_12')
df = trend.calculate_ema(df, period=26, output_column='EMA_26')

print("Medias móviles calculadas:")
print(df[['close', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26']].tail())

# Visualizar
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index, df['close'], label='Precio', linewidth=1.5, alpha=0.7)
ax.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=2)
ax.plot(df.index, df['SMA_50'], label='SMA 50', linewidth=2)
ax.plot(df.index, df['EMA_12'], label='EMA 12', linewidth=2, linestyle='--')
ax.plot(df.index, df['EMA_26'], label='EMA 26', linewidth=2, linestyle='--')
ax.set_title(f'{symbol} - Medias Móviles', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha')
ax.set_ylabel('Precio ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Indicadores de Momentum

### RSI

In [ ]:
# Calcular RSI
df = momentum.calculate_rsi(df, period=14, output_column='RSI_14')

print(f"RSI actual: {df['RSI_14'].iloc[-1]:.2f}")

# Visualizar RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, height_ratios=[2, 1])
ax1.plot(df.index, df['close'], label='Precio', linewidth=1.5, color='black')
ax1.set_title(f'{symbol} - Precio', fontsize=12, fontweight='bold')
ax1.set_ylabel('Precio ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(df.index, df['RSI_14'], label='RSI 14', linewidth=2, color='purple')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Sobrecompra (70)')
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Sobreventa (30)')
ax2.fill_between(df.index, 70, 100, alpha=0.2, color='red')
ax2.fill_between(df.index, 0, 30, alpha=0.2, color='green')
ax2.set_title('RSI (Relative Strength Index)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Fecha')
ax2.set_ylabel('RSI')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Indicadores de Volatilidad

### Bollinger Bands

In [ ]:
# Calcular Bollinger Bands y ATR
df = volatility.calculate_bollinger_bands(df, period=20, std_dev=2.0, output_prefix='BB_20')
df = volatility.calculate_atr(df, period=14, output_column='ATR_14')

print(f"ATR actual: ${df['ATR_14'].iloc[-1]:.2f}")

# Visualizar Bollinger Bands
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index, df['close'], label='Precio', linewidth=1.5, color='black')
ax.plot(df.index, df['BB_20_Upper'], label='Banda Superior', linewidth=1.5, color='red', alpha=0.7)
ax.plot(df.index, df['BB_20_Middle'], label='Banda Media', linewidth=1.5, color='blue', alpha=0.7)
ax.plot(df.index, df['BB_20_Lower'], label='Banda Inferior', linewidth=1.5, color='green', alpha=0.7)
ax.fill_between(df.index, df['BB_20_Upper'], df['BB_20_Lower'], alpha=0.1, color='gray')
ax.set_title(f'{symbol} - Bollinger Bands', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha')
ax.set_ylabel('Precio ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Indicadores de Volumen

### VWAP y MFI

In [ ]:
# Calcular VWAP y MFI
df = volume.calculate_vwap(df, output_column='VWAP')
df = volume.calculate_mfi(df, period=14, output_column='MFI_14')

print(f"VWAP actual: ${df['VWAP'].iloc[-1]:.2f}")
print(f"MFI actual: {df['MFI_14'].iloc[-1]:.2f}")

# Visualizar VWAP
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index, df['close'], label='Precio', linewidth=1.5, alpha=0.7)
ax.plot(df.index, df['VWAP'], label='VWAP', linewidth=2, color='orange')
ax.set_title(f'{symbol} - VWAP', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha')
ax.set_ylabel('Precio ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Resumen

Todos los indicadores han sido calculados y añadidos al DataFrame. Estos indicadores pueden usarse como base para análisis más avanzados y estrategias (implementadas en módulos separados).